In [5]:
from fim import apriori, eclat, fpgrowth, fim
import numpy as np
import pandas as pd
from keras.datasets import fashion_mnist, mnist
import matplotlib.pyplot as plt

from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import binarize, OneHotEncoder


# Data preparation

In [ ]:
mnist_array = np.loadtxt("mnist_8x8_image.csv", delimiter=',')
mnist_array = mnist_array.astype(np.int64)
col_names = [str(i) for i in (range(mnist_array.shape[1]))]
mnist_df = pd.DataFrame(mnist_array, columns=col_names)
mnist_df.head()

In [ ]:
mnist_label = np.loadtxt("mnist_8x8_label_onehot.csv", delimiter=",")
col_names = [str(64 + i) for i in range(mnist_label.shape[1])]
mnist_label = pd.DataFrame(mnist_label, columns=col_names)

mnist_label.head(10)

In [ ]:
mnist_label_int = np.argmax(mnist_label.values, axis=1)
mnist_label_int = pd.DataFrame(mnist_label_int, columns=['label'])
mnist_label_int.head()
mnist_label_int.label.value_counts()

### concat (image + label) dataframe

In [ ]:
mnist_conc = pd.concat([mnist_df, mnist_label], axis=1)
mnist_conc.head()

# Data conversion

table -> transaction 

In [3]:
def pickupColumns(df, ind):
    colnames = df.columns
    return  list(np.array(colnames[df.iloc[ind,:] ==1],dtype=np.int64))

In [ ]:
transactions = [pickupColumns(mnist_df,i) for i in range(mnist_df.shape[0])]
transactions = np.array(transactions)

transactions_conc = [pickupColumns(mnist_conc,i) for i in range(mnist_conc.shape[0])]
transactions_conc = np.array(transactions_conc)
transactions_conc


# frequent items in mnist_array

In [ ]:
%time res = np.array(fpgrowth(transactions, supp=10, zmin=7, report='s'))

freq_df = pd.DataFrame(res, columns=['itemsets', 'support'])
freq_df = freq_df.sort_values('support', ascending=False)
freq_df.index = np.arange(freq_df.shape[0])
freq_df

## association rules in mnist_conc

In [ ]:
%time res = np.array(fpgrowth(transactions_conc, conf=40, supp=3, zmin=5, report='C', target='r', eval='x', agg='m', thresh=10))

freq_df = pd.DataFrame(res, columns=['consequent','antecedent', 'confident'])
freq_df = freq_df.sort_values('confident', ascending=False)
freq_df.index = np.arange(freq_df.shape[0])
print(freq_df.shape)
#freq_df

In [ ]:
rules = freq_df[freq_df.consequent >=64]
max_rules = rules.groupby(rules['consequent']).count()
max_rules

In [ ]:
import  matplotlib.pyplot as plt 
row = 10
col = 6

_, axes = plt.subplots(row,col, figsize = (col*1.3,row*1.3))


for i in range(row):
    r = rules[rules.consequent==(i+64)]
    r_best5 = r[:col]
    for j in range(col):
        ax = axes[i][j]
        image = np.zeros(64)
        image[list(r_best5.iloc[j,:].antecedent)] =1
        ax.set_axis_off()
        ax.imshow(image.reshape(8,8), cmap=plt.cm.gray_r, interpolation='nearest')

    

In [ ]:
%debug

In [ ]:
for i in range(10):
    print(i)
    print(rules[rules.consequent==(i+64)])
    print()

In [ ]:
transactions = [pickupColumns(mnist_df,i) for i in range(mnist_df.shape[0])]
transactions = np.array(transactions)

%time res = np.array(fpgrowth(transactions, supp=5, zmin=10, report='s'))

freq_df = pd.DataFrame(res, columns=['itemsets', 'support'])
freq_df = freq_df.sort_values('support', ascending=False)
freq_df.index = np.arange(freq_df.shape[0])
freq_df.head()

In [ ]:
transactions = [pickupColumns(mnist_df,i) for i in range(mnist_df.shape[0])]
transactions = np.array(transactions)

%time res = fpgrowth(transactions, supp=5, zmin=10, report='s', target='r')
res
freq_df = pd.DataFrame(res, columns=['consequent','antecedent', 'support'])
freq_df = freq_df.sort_values('support', ascending=False)
freq_df.index = np.arange(freq_df.shape[0])
freq_df.head()

In [ ]:
np.array(res)

In [ ]:
transactions = [pickupColumns(mnist_df,i) for i in range(mnist_df.shape[0])]
transactions = np.array(transactions)

%time res = np.array(fim(transactions, supp=5, zmin=10, report='s'))

freq_df = pd.DataFrame(res, columns=['itemsets', 'support'])
freq_df = freq_df.sort_values('support', ascending=False)
freq_df.index = np.arange(freq_df.shape[0])
freq_df.head()

In [ ]:
freq_df

## fashion mnist

In [4]:
!mkdir data/fashion_mnist

mkdir: data/fashion_mnist: File exists


In [ ]:
((fashion_mnist_trainX, fashion_mnist_trainY), (fashion_mnist_testX, fashion_mnist_testY)) = fashion_mnist.load_data()


In [ ]:
fashion_mnist_trainX = trainX
fashion_mnist_trainY = trainY
fashion_mnist_testX = testX
fashion_mnist_testY = testY

In [ ]:
np.savez('fashion_mnist_trainX.npz', trainX)
np.savez('fashion_mnist_trainY.npz', trainY)
np.savez('fashion_mnist_testX.npz', testX)
np.savez('fashion_mnist_testY.npz', )

In [ ]:
!mv fashion_mnist*.npz data/fashion_mnist/

In [ ]:
row = 6
col = 15
_, axes = plt.subplots(row,col, figsize=(15,12))
for i in range(row*col):
    r = int(i / col)
    c = i % col
    ax = axes[r][c]
    image = fashion_mnist_trainX[i]
    label = fashion_mnist_trainY[i]
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title('%i' % label)
    
del image, label

### binarize

In [1]:
threshold = 96
#fm_bin = binarize(fashion_mnist_trainX.reshape(60000,-1), threshold)
fm_bin = binarize(fashion_mnist_trainX, threshold)

NameError: name 'binarize' is not defined

In [ ]:
row = 6
col = 15
_, axes = plt.subplots(row, col, figsize=(15,12))
for i in range(row*col):
    r = int(i / col)
    c = i % col
    ax = axes[r][c]
    image = fm_bin
    label = fashion_mnist_trainY[i]
    ax.set_axis_off()
    ax.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    ax.set_title('Training: %i' % label)
    
del image, label

In [ ]:
transactions = [pickupColumns(mnist_df,i) for i in range(mnist_df.shape[0])]
transactions = np.array(transactions)

transactions_conc = [pickupColumns(mnist_conc,i) for i in range(mnist_conc.shape[0])]
transactions_conc = np.array(transactions_conc)
transactions_conc
